# 4강) TF-IDF를 활용한 Passage Retrieval 실습

### Requirements

In [1]:
# !pip install datasets
# !pip install transformers

## 데이터셋 준비

KorQuAD train 데이터셋을 search corpus로 활용

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (/opt/ml/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


In [3]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비

가장 기본적인 띄워쓰기를 기준으로 token을 나누는 tokenizer를 활용 (성능 향상을 위해 더 세밀한 토크나이저 활용 가능)

In [4]:
tokenizer_func = lambda x: x.split(' ')

In [5]:
tokenizer_func(corpus[0])[:10]

['세종실록지리지에',
 '따르면,',
 '남한산성',
 '내부에는',
 '조선초기',
 '까지만',
 '하더라도',
 '124결이라는',
 '좁지',
 '않은']

## TF-IDF embedding 만들기

Scikit-learn의 TfidfVectorizer를 활용하여 TF-IDF embedding 만들어보기 
(unigram, bigram 활용)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenizer_func, ngram_range=(1,2))

In [7]:
# corpus 에 있는 단어들을 vectorize 가 갖고 있어야 함. fit 을 통해서 corpus에 있는 것들을 vectorized 시킨다.
vectorizer.fit(corpus)
sp_matrix = vectorizer.transform(corpus)

/opt/conda/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [8]:
sp_matrix.shape

(9606, 1272768)

In [9]:
import pandas as pd
df = pd.DataFrame(sp_matrix[0].T.todense(), index=vectorizer.get_feature_names(), columns=["TF-IDF"])
df = df.sort_values('TF-IDF', ascending=False)
print(df.head(10))

               TF-IDF
그다지 높지       0.174671
경지가          0.174671
높지           0.143251
그다지          0.122452
수 있다.        0.100980
알 수          0.096236
알            0.090450
때,           0.089071
면적의 경지가      0.087335
산성내의 토지부양력이  0.087335


## TF-IDF embedding을 활용하여 passage retrieval 실습해보기

앞서 만든 sparse embedding을 활용하여, 실제 passage retrieval 수행해보기

Search query로 KorQuAD train 데이터셋의 질문을 활용하여, 실제 context가 잘 나오는지 확인해보기

In [10]:
import random
import numpy as np

random.seed(1)
sample_idx = random.choice(range(len(dataset['train'])))

query = dataset['train'][sample_idx]['question']
ground_truth = dataset['train'][sample_idx]['context']

Query를 tf-idf vector로 변환

In [11]:
query_vec = vectorizer.transform([query])

In [12]:
query_vec.shape

(1, 1272768)

변환된 query vector를 document들의 vector과 dot product를 수행 => Document들의 similarity ranking을 구함

In [13]:
result = query_vec * sp_matrix.T
result.shape

(1, 9606)

In [33]:
result.data

array([0.02374371, 0.02095962, 0.02427458, 0.03065025, 0.02792934,
       0.0087434 , 0.00515867, 0.01739697, 0.00807631, 0.00771468,
       0.00895329, 0.00554998, 0.00621966, 0.00789726, 0.00895742,
       0.009516  , 0.00766781, 0.00726265, 0.00777131, 0.00861243,
       0.0083884 , 0.00690264, 0.00840719, 0.0069108 , 0.00867212,
       0.00804823, 0.0085508 , 0.00650207, 0.00869146, 0.01419991,
       0.00762781, 0.00936421, 0.00593566, 0.00718984, 0.0058665 ,
       0.0052843 , 0.00590311, 0.0088058 , 0.00722671, 0.00782319,
       0.00485716, 0.00621068, 0.00725701, 0.00919418, 0.01879939,
       0.00714483, 0.00869661, 0.00770822, 0.00798599, 0.00867108,
       0.00657925, 0.00950598, 0.0079862 , 0.00920435, 0.00815161,
       0.0076981 , 0.00816605, 0.00789206, 0.01274595, 0.00548186,
       0.00831748, 0.00748837, 0.0154252 , 0.00636568, 0.0086421 ,
       0.0066185 , 0.00834802, 0.00737311, 0.00366208, 0.00652947,
       0.00570515, 0.00800105, 0.00630131, 0.00843764, 0.00844

In [14]:
sorted_result = np.argsort(-result.data)
doc_scores = result.data[sorted_result]
doc_ids = result.indices[sorted_result]

In [35]:
result.indices

array([5891, 3098, 6792,  892, 3910, 9544, 9533, 9462, 9313, 9278, 9229,
       9167, 9133, 9108, 9084, 9061, 9005, 8986, 8967, 8958, 8945, 8868,
       8850, 8845, 8815, 8711, 8710, 8679, 8572, 8530, 8512, 8495, 8489,
       8463, 8453, 8328, 8294, 8285, 8278, 8239, 8210, 8188, 8187, 8160,
       8099, 8053, 8026, 7988, 7986, 7949, 7946, 7896, 7865, 7609, 7572,
       7560, 7397, 7381, 7372, 7367, 7361, 7343, 7325, 7304, 7300, 7275,
       7196, 7129, 7126, 7124, 7112, 6989, 6948, 6908, 6823, 6679, 6598,
       6593, 6562, 6532, 6464, 6414, 6324, 6270, 6255, 6226, 6004, 5954,
       5949, 5916, 5884, 5870, 5822, 5682, 5636, 5616, 5569, 5547, 5347,
       5336, 5291, 5071, 5042, 4945, 4939, 4903, 4893, 4861, 4838, 4800,
       4763, 4756, 4751, 4668, 4582, 4540, 4534, 4489, 4312, 4243, 4207,
       4125, 4109, 4076, 4060, 4041, 3983, 3948, 3927, 3924, 3877, 3875,
       3869, 3838, 3794, 3766, 3598, 3574, 3534, 3468, 3464, 3452, 3310,
       3291, 3284, 3236, 3205, 3059, 3052, 3018, 29

In [34]:
sorted_result

array([360, 313, 351,   3, 124,  94,   4, 300,   2,   0, 238, 357, 353,
       316,   1, 358, 224, 321, 265, 268, 315,  44, 308, 355, 359, 350,
       335,   7, 198, 352, 354, 248, 199, 294, 356,  62, 225, 119, 282,
        29, 105, 290, 136,  58, 331, 304, 226, 230, 217, 333, 319, 278,
       342, 310, 229, 211, 245, 266, 292, 269, 149, 246, 215, 210, 242,
       284, 261, 257, 219, 305, 279, 222, 343, 320, 329, 243, 323, 208,
       233, 250, 340, 256, 129, 274, 277, 314, 309, 280, 258, 299, 271,
       325, 337,  15, 212,  51, 327, 334, 145, 322, 345, 197, 214, 285,
        31, 166, 260, 154, 200, 324, 348, 155, 249, 341,  53, 132, 244,
        43, 173, 247, 306, 192, 220, 275, 298, 122, 262, 138, 158, 273,
       189, 150,  86, 328, 116, 207, 272, 338,  14,  10, 286, 318, 121,
        96, 297,  87, 291, 152, 188, 137,  37, 263, 163, 170, 115, 107,
       239,   5, 254, 326,  46,  28, 346,  24,  49,  64, 332,  19, 139,
       209, 288, 236, 168,  26,  78, 159, 347, 185, 251, 194, 33

Top-3개의 passage를 retrieve 하고, 실제 ground_truth와 비교해보기

In [15]:
k = 3
doc_scores[:k], doc_ids[:k]

(array([0.18985967, 0.03625019, 0.03371167]),
 array([5623, 2707, 8917], dtype=int32))

In [16]:
print("[Search query]\n", query, "\n")

print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
  doc_id = doc_ids[i]
  print(corpus[doc_id], "\n")


[Search query]
 호메로스 찬가를 신통기에 비해 간결한 서사로 간주한 사람은 누구인가? 

[Ground truth passage]
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-1 passage with score 0.1899
고전 시대 신화에서는 티탄들의 패배 이후, 신들의 새로운 판테온이 세워졌다고 설명한다. 주요한 그리스 신들 중에서 올림피안은 올림포스 산 정상에서 제우스의 통치 아래 살아가는 신들을 말한다. 이들의 인원이 열두 명으로 제한된 것은 비교적 최근에 도입된 개념으로 보인다. 올림피안 이외에도 그리스인들은 염소 신 판, 강의 정령 님프, 샘에 사는 나이아드, 나무의 정령 드라이어드, 바다에 사는 네레이드, 강의 신, 사티로스를 비롯한 그 지역의 다양한 신들을 숭배하였다. 여기에는 에리니에스(또는 푸리아이)처럼 혈연 관계에게 범죄를 저지른 죄인을 뒤쫓는 저승의 암흑 세력도 있었다. 시인들은 그리스 판테온의 영광을 기리고자 호메로스 찬가를 지었다.(33편의 노래). 그레고리 나지는 호메로스 찬가를 "각 노래마다 신에 대한 기원을 노래하는(《신통기》에 비해) 간결한 서가"로 간주하였다. 

Top-2 